In [75]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window as W

In [26]:
spark = SparkSession.builder.appName('test_name').getOrCreate()

In [32]:
df = spark.read.option('inferSchema', 'true')\
.option('header', 'true').csv("/users/dznosok/downloads/owid-covid-data.csv")

In [33]:
df.printSchema()

root
 |-- iso_code: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- location: string (nullable = true)
 |-- date: date (nullable = true)
 |-- total_cases: double (nullable = true)
 |-- new_cases: double (nullable = true)
 |-- new_cases_smoothed: double (nullable = true)
 |-- total_deaths: double (nullable = true)
 |-- new_deaths: double (nullable = true)
 |-- new_deaths_smoothed: double (nullable = true)
 |-- total_cases_per_million: double (nullable = true)
 |-- new_cases_per_million: double (nullable = true)
 |-- new_cases_smoothed_per_million: double (nullable = true)
 |-- total_deaths_per_million: double (nullable = true)
 |-- new_deaths_per_million: double (nullable = true)
 |-- new_deaths_smoothed_per_million: double (nullable = true)
 |-- reproduction_rate: double (nullable = true)
 |-- icu_patients: double (nullable = true)
 |-- icu_patients_per_million: double (nullable = true)
 |-- hosp_patients: double (nullable = true)
 |-- hosp_patients_per_million: 

Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [52]:
df.select('iso_code', 'location', (F.col('total_cases')/F.col('population')).alias('procent'))\
.where(F.col('date') == '2020-03-31')\
.orderBy(F.col('procent').desc())\
.show(15)

+--------+--------------+--------------------+
|iso_code|      location|             procent|
+--------+--------------+--------------------+
|     VAT|       Vatican|0.007416563658838072|
|     SMR|    San Marino|0.006953857033413872|
|     AND|       Andorra|0.004866368989840161|
|     LUX|    Luxembourg|0.003479366621084...|
|     ISL|       Iceland|0.003326007326007326|
|     ESP|         Spain|0.002051618975538...|
|     CHE|   Switzerland|0.001918628875358...|
|     LIE| Liechtenstein|0.001783045336549...|
|     ITA|         Italy|0.001749732078891...|
|     MCO|        Monaco|0.001325043318723...|
|     AUT|       Austria|0.001130307337004...|
|     BEL|       Belgium|0.001102279833947...|
|OWID_EUN|European Union|8.938996679530879E-4|
|     DEU|       Germany|8.570615766540952E-4|
|     NOR|        Norway|8.560768916052816E-4|
+--------+--------------+--------------------+
only showing top 15 rows



Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [65]:
df.select('date', 'location', 'new_cases')\
.where(F.col('date') >= '2021-03-29').where(F.col('date') <= '2021-04-04')\
.where(F.col('continent') != 'null')\
.orderBy(F.col('new_cases').desc()).show(10)


+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-04-04|        India| 103558.0|
|2021-04-03|        India|  93249.0|
|2021-04-01|       Brazil|  91097.0|
|2021-03-31|       Brazil|  90638.0|
|2021-04-02|        India|  89129.0|
|2021-03-30|       Brazil|  84494.0|
|2021-04-01|        India|  81466.0|
|2021-04-04|       France|  80629.0|
|2021-04-01|United States|  79045.0|
|2021-03-31|        India|  72330.0|
+----------+-------------+---------+
only showing top 10 rows



Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)



In [91]:
window = W.orderBy("date")

In [101]:
df.select( 'date', 'new_cases', F.col('new_cases') - F.col('new_cases'))\
.where(F.col('location') == 'Russia')\
.select('date', F.lag('new_cases').over(window).alias('lag'),'new_cases')\
.where(F.col('date') >= '2021-03-29').where(F.col('date') <= '2021-04-04')\
.select('date', 'lag', 'new_cases', (F.col('new_cases')-F.col('lag')).alias('delta')).show()

+----------+------+---------+------+
|      date|   lag|new_cases| delta|
+----------+------+---------+------+
|2021-03-29|8979.0|   8589.0|-390.0|
|2021-03-30|8589.0|   8162.0|-427.0|
|2021-03-31|8162.0|   8156.0|  -6.0|
|2021-04-01|8156.0|   9057.0| 901.0|
|2021-04-02|9057.0|   8682.0|-375.0|
|2021-04-03|8682.0|   8906.0| 224.0|
|2021-04-04|8906.0|   8697.0|-209.0|
+----------+------+---------+------+



23/10/31 22:47:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/31 22:47:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/31 22:47:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/31 22:47:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/31 22:47:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [102]:
spark.stop()